In [24]:
import pandas as pd
import numpy as np
import datetime
import pickle

In [25]:
test_short= pd.read_pickle("./../data/processed/test_short.pkl")
test_long= pd.read_pickle("./../data/processed/test_long.pkl")

In [27]:
head=["id","date_time","flag","option"]

temporal_dataset = pd.read_csv("./../data/FTDD/temporal_activity.csv",header=None)
temporal_dataset.columns=head
temporal_dataset["date_time"]=pd.to_datetime(temporal_dataset["date_time"])

In [28]:
temporal_dataset.head()

,id,date_time,flag,option
0,322067,2006-01-01 04:10:56,N,0
1,322067,2007-06-06 15:24:00,W,521
2,322067,2007-06-06 15:24:00,C,521
3,322067,2007-07-04 14:35:00,Z,549
4,322075,2006-01-01 06:08:45,N,0


In [29]:
observation_dict = {'N':0,'M':1,'E':2,'A':3,'R':4,'C':5,'D':6,'V':7,'Y':8,'S':9,'H':10,'F':11,'W':12,'L':13,'P':14,'Q':15,'Z':16}


            

In [30]:
for n_days in [1,4,8]:

    test_short["day"+str(n_days)] = ""

    for index, row in temporal_dataset.iterrows():
        start_temp = test_short.loc[test_short['id'] == row["id"], 'start']
        if len(start_temp)!=0:
            if ((row["date_time"]-start_temp.iloc[0]).days) < n_days:
                test_short.loc[test_short['id'] == row["id"], "day"+str(n_days) ]+= row["flag"] + " "

    test_long["day"+str(n_days)] = ""

    for index, row in temporal_dataset.iterrows():
        start_temp = test_long.loc[test_long['id'] == row["id"], 'start']
        if len(start_temp)!=0:
            if ((row["date_time"]-start_temp.iloc[0]).days) < n_days:
                test_long.loc[test_long['id'] == row["id"], "day"+str(n_days) ]+= row["flag"] + " "


    test_long["day"+str(n_days)] = test_long["day"+str(n_days)].apply(lambda x: x.strip().split())
    test_long["day"+str(n_days)] = test_long["day"+str(n_days)].apply(lambda x: list(map(lambda y: int(observation_dict[y]),x)) )


    test_short["day"+str(n_days)] = test_short["day"+str(n_days)].apply(lambda x: x.strip().split())
    test_short["day"+str(n_days)] = test_short["day"+str(n_days)].apply(lambda x: list(map(lambda y: int(observation_dict[y]),x)) )


            

In [31]:
test_long.to_pickle("./../data/processed/test_long_temp1.pkl")
test_short.to_pickle("./../data/processed/test_short_temp1.pkl")

In [32]:
test_short= pd.read_pickle("./../data/processed/test_short_temp1.pkl")
test_long= pd.read_pickle("./../data/processed/test_long_temp1.pkl")

In [33]:
#len(compressed_set.flag)
len_sequence_long_test=[len(xi) for xi in test_long.day1]
sequence_long_test=[[[int(x)] for x in xi]  for xi in test_long.day1]


#len(compressed_set.flag)
len_sequence_short_test=[len(xi) for xi in test_short.day1]
sequence_short_test=[[[int(x)] for x in xi]  for xi in test_short.day1]

In [34]:
from evaluation import first_week_number_testing

In [35]:
first_week_temporal_sequence_result=pd.DataFrame([],columns=["Number of days","Number of States","Precision","Recall","F-measure","Accuracy"])
columns = list(first_week_temporal_sequence_result)
data = []

In [36]:
for n_days in [1,4,8]:

    #len(compressed_set.flag)
    len_sequence_long_test=[len(xi) for xi in test_long["day"+str(n_days)]]
    sequence_long_test=[[[int(x)] for x in xi]  for xi in test_long["day"+str(n_days)]]


    #len(compressed_set.flag)
    len_sequence_short_test=[len(xi) for xi in  test_short["day"+str(n_days)]]
    sequence_short_test=[[[int(x)] for x in xi]  for xi in  test_short["day"+str(n_days)]]



    for n_states in [5,10,15]:


        fname="./models/hmm_long_"+str(n_states)+".pkl"
        with open(fname, 'rb') as input:
            hmm_long = pickle.load(input)
        fname="./models/hmm_short_"+str(n_states)+".pkl"
        with open(fname, 'rb') as input:
            hmm_short = pickle.load(input)

        tp,tn,fn,fp=first_week_number_testing(sequence_short_test,sequence_long_test,hmm_short,hmm_long)
        precision=tp/(tp+fp)
        recall=tp/(tp+fn)
        accuracy=(tp+tn)/(tp+tn+fn+fp)
        f1 = (2*precision*recall)/(precision+recall)
        values =[n_days ,n_states,precision,recall,f1,accuracy]
        zipped = zip(columns, values)
        a_dictionary = dict(zipped)
        data.append(a_dictionary)



In [37]:
first_week_temporal_sequence_result = first_week_temporal_sequence_result.append(data,True)

In [38]:
first_week_temporal_sequence_result

,Number of States,Precision,Recall,F-measure,Accuracy
0,5,0.454827,0.470995,0.462770,0.616981
1,10,0.517072,0.735209,0.607142,0.666753
2,15,0.518523,0.687567,0.591199,0.666955
3,5,0.481893,0.478483,0.480182,0.637155
4,10,0.561785,0.771744,0.650236,0.709205
5,15,0.559604,0.725418,0.631813,0.703873
6,5,0.498538,0.491072,0.494777,0.648741
7,10,0.574446,0.780959,0.661970,0.720647
8,15,0.571218,0.734880,0.642795,0.713932
